In [4]:
# импорт библиотек для работы программы

import pandas as pd
import os
import random
import string
import smtplib
from email.mime.text import MIMEText

# Если в этой ячейке будут ошибки, то значит, что какой-то библиотеки у тебя на компьютере нет
# Нужно установить ее, выполнив в терминале (программа в маке) команду, например:
# pip3 install smtplib

## Ячейка настроек. Устанавливать настройки только в ней

In [5]:
# Укажи путь до .xlsx файла с имейлами
# в файле должен быть один лист, с одной колонкой email и списком адресов

input_path = r"/Users/vitaly/Desktop/mails.xlsx"

# Укажи адрес эксельки с выводом (выведутся адреса и сгенерированные пароли)
output_path = r"/Users/vitaly/Desktop/mails_n_pass.xlsx"

# Отправлять письма? 1 если да, 0 если нет. Если нет, то просто сгенерируем эксельку с паролями
send_email = 1

# Длина пароля
pass_length = 8

# почта, с которой нужно отправить сообщение 
your_email = 'yourmail@gmail.com'
# пароль к почте для приложений (это не тот же самый пароль, что ты использешь для входа)
# гугл запрещает доступ автоматически по тому же паолю, по которому входит человек
# поэтому нужно для своей почты сгенерировать пароль для приложений
# получить паролль для приложений можно здесь (5 мин)
# https://support.google.com/accounts/answer/185833?visit_id=637981830698552150-915730886&p=InvalidSecondFactor&rd=1
your_password = 'yourgeneratedpass'

# Вообще отправка писем это немного индивидуальная вещь
# я сделал свой пример для gmail
# у него вот такие настройки:
smtp_server = "smtp.gmail.com"
port = 587
# если будешь пользоваться корп почтой, то они будут другими.
# Здесь нужна будет помощь сисадминов, чтоб они подсказали с сервером, портом


# После установки всех настроек нажми Kernel -> Restart and run all
# 1) Первый раз прогони с send_email = 0, чтобы убедиться, что эксельки читаются и пишутся
# 2) Второй раз прогони со своей почтой в файлике с получателями, для примера.
# 3) Если все ок - запускай с полным списком получателей

## Процесс

In [6]:
# Прочитаем файл с почтой

mails = pd.read_excel(input_path)

print(f'В файле {len(mails)} адреса(ов)')

В файле 1 адреса(ов)


In [7]:
# Сгенерируем пароль для каждого адреса

# функция генерации пароля
def generate_pass(pass_length):
    letters = string.ascii_lowercase + string.digits
    pswrd = ''.join(random.choice(letters) for i in range(pass_length))
    return pswrd

# Для каждой почты сгенерируем пароли
mails['pass'] = [generate_pass(pass_length) for row in mails.index]

# Попутно сохраним файл с паролями, чтоб не потерять
mails.to_excel(output_path)

In [8]:
# Посмотрим на первые пять строчек получившегося набора 

mails.head(5)

,email,pass
0,zederblatt@gmail.com,79qxw0fl


In [9]:
# функция по отправке сообщения
def emailing(your_email, your_password, email_to, subject, message
             , smtp_server = "smtp.gmail.com"
             , port = 587):

    server = smtplib.SMTP(host = smtp_server, port = port)
    server.starttls()
    server.login(your_email, your_password)
    
    msg = MIMEText(message)
    msg['Subject'] = subject
    msg['From'] = your_email
    msg['To'] = email_to

    server.sendmail(your_email, [email_to], msg.as_string())
    server.quit()

    
# Непосредственно запуск функции и отправка сообщения
if send_email == 1:
    print('Начинаем отправлять сообщения!')
    for row in mails.reset_index().index:
        try:
            emailing(your_email=your_email, your_password=your_password
                   , email_to=mails.loc[row, 'email']
                   , subject="пароль от расчетного листка"
                   , message=f"ваш пароль от расчетного листка: {mails.loc[row, 'pass']}"
                    )
            print(f"{row}: сообщение на {mails.loc[row, 'email']} с паролем {mails.loc[row, 'pass']} отправлено")
        except Exception as e:
            print(f"""{row} ВНИМАНИЕ: сообщение на {mails.loc[row, 'email']} с паролем {mails.loc[row, 'pass']} НЕ отправлено!
            по причине: {e}""") 
else:
    print('Сообщения не будут отправлены, т.к. send_email = 0')

Начинаем отправлять сообщения!
0: сообщение на zederblatt@gmail.com с паролем 79qxw0fl отправлено
